In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
from langchain_community.graphs import Neo4jGraph

load_dotenv()


graph = Neo4jGraph(
    url="neo4j://127.0.0.1:7687",
    username="neo4j",
    password="12345678",
    refresh_schema=False,
    database="mark1",
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=os.getenv("GEMINI_API_KEY")
)



allowed_nodes = [
    "Person",
    "Company",
    "Product",
    "Technology",
    "Event",
    "Sector"
]

allowed_relationships = [
    "WORKS_AT",
    "LEADS",
    "DEVELOPS",
    "INVESTS_IN",
    "PARTNERS_WITH",
    "SUPPLIES",
    "COMPETES_WITH",
    "UTILIZES",
    "AFFECTS",    
    "REGULATES"
]
graph_transformer_nodes_defined = LLMGraphTransformer(llm=llm, 
                                                      allowed_nodes=allowed_nodes,
                                                      allowed_relationships=allowed_relationships
                                                    )

directory = "data"
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        filepath = os.path.join(directory, filename)
        with open(filepath, "r", encoding="utf-8") as file:
            text = file.read()
        
        documents = [Document(page_content=text)]
        graph_documents_nodes_defined = await graph_transformer_nodes_defined.aconvert_to_graph_documents(documents)
        graph.add_graph_documents(graph_documents_nodes_defined, include_source=True)
